# 장고 웹 프로그래밍 실습

실습 환경 : Pycharm, python 3.6 ver
책 & 예제 : 파이썬 웹프로그래밍 chapter 3(개정판, 한빛미디어)

## 1. 프로젝트 생성
터미널에서 처음부터 프로젝트를 생성하자

```
> django-admin startproject mysite(프로젝트 이름)
```
프로젝트 이름의 디렉토리가 위 아래로 생겨서 이름을 바꿔줘도 상관없다.
```
> move myProject(새로운 이름)
```

## 2. 애플리케이션 생성
프로젝트 루트 디렉토리에서 polls라는 애플리케이션을 만든다.

```
> python manage.py startapp polls(어플리케이션 이름)
```
어플리케이션 디렉토리와 하위 파일들이 자동으로 생성되었음을 알 수 있다.
개발자들은 알아서 그 내요을 채워넣기만 하면 된다.

## 3. 프로젝트 설정 파일 변경
프로젝트에 필요한 설정은 settinfs.py에 있다.\
루트 디렉토리를 포함한 각종 디렉토리의 위치, 로그의 형식, 프로젝트에 포함된 애플리케이션의 이름 등이 지정되어 있다. 다음은 프로젝트 설정 파일을 변경하는 순서이다.

1. ALLOWED_HOSTS 항목을 지정해야 한다. 장고는 DEBUG=True이면 개발 모드이며 False는 운영모드이다. 운영모드인 경우 반드시 서버의 IP나 도메인을 지정해야 하고, 개발 모드인 경우에는 ['localhost','127.0.0.1']로 간주한다.\
개발모드에서 서버 IP가 기존과 다를 수 있다면 아래와 같이 지정한다.

```python
ALLOWED_HOSTS = [ '192.168.56.101', 'localhost', '127.0.0.1' ]
```

2. 프로젝트에 포함되는 애플리케이션은 모두 설정 파일에 등록되어야 한다. 따라서 polls도 등록해야한다. 정확히는 애플리케이션의 설정 클래스로 등록을 하면 된다.\
polls앱의 설정 클래스는 startapp polls 명령 시에 생성된 apps.py에 PollsConfig로 정의되어 있다. 장고가 설정을 찾을 수 있도록 모듈 경로가지 포함하여 등록하자

```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'polls.apps.PollsConfig', # 추가되었다.
]
```

3. 프로젝트에 사용할 DB엔진을 설정한다. 장고는 default로 SQLite3를 사용하고 있다. 이 외의 다른 DB를 사용하고 싶다면 settings.py에서 수정하면 된다.
```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}
```

4. 타임존 지정을 한다. default는 세계표준시(UTC)이며 한국 시간으로 바꿔준다.
```python
TIME_ZONE = 'Asia/Seoul'
```

## 4. 기본 테이블 생성
기본 테이블을 생성한다. migrate 명령은 DB의 변경사항이 있을 때 이를 반영해주는 명령어이다.
```
>python manage.py migrate
```
아직 테이블을 만들지는 않았으나 장고는 모든 웹 프로젝트 개발 시 반드시 user와 그룹 테이블 등이 필요하다는 가정 하에 설계되었다.\
쉽게 말해서 실질적 데이터가 없더라도 빈 DB파일이 처음부터 필요하다는 뜻 같다.

## 5. 지금까지의 작업 확인하기
현재까지 프로젝트 디렉토리, 애플리케이션 디렉토리를 비롯한 관련 파일들 그리고 user 및 그룹 테이블을 만들었다.\

이를 확인하기 위해서 웹 서버를 실행하고, 접속할 수 있다.

```
>python manage.py runserver
```

터미널에서 나타난 주소(http://127.0.0.1:8000/)에 접속하면 장고 기본 화면이 뜬다.\
이제 테이블을 확인해보자. 로그인을 하기 위하여 계정을 만든다.

```
>python manage.py createsuperuser
```
명령어 실행 이후 나타난 지시에 따라 계정을 생성한다.\
(비밀번호는 일부러 안보여준다 잘 기억하자)\
\
계정을 만들고 난 후 http://127.0.0.1:8000/admin 에 접속하면 테이블을 확인할 수 있다.\
앞으로 이 사이트에서 user와 group 테이블을 포함하여 데이터를 조작할 수 있다.

# Model 코딩

모델 작업은 DB에 테이블을 생성하는 작업이다.\
모델링은 책의 예제대로 따라간다. 진행은 다음 순서와 같이 진행된다.

## 1. 테이블 정의
polls 애플리케이션은 Question과 Choice 두 개의 테이블이 필요하다(예제의 요구사항이다).\
테이블은 models.py에 정의한다. 

In [ ]:
# models.py
from django.db import models 
class Questions(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField('date published')
    
    # 객체를 문자열로 표현할 때 사용한다. Admin사이트나 장고 쉘 등에서 
    # 테이블명을 제대로 표현하기 위해서 필수적이다
    def __str__(self):
        return self.question_text

class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)
    
    def __str__(self):
        return self.choice_text


장고에서는 테이블을 하나의 클래스로 정의하고, column은 클래스의 변수로 매핑한다. 테이블 클래스는 django.db.models.Model을 상속받으며 각 변수의 타입도 미리 정의된 필드 클래스를 사용한다.

* Question 테이블 컬럼과 클래스 변수 간 매핑

| 테이블 컬럼명 | 컬럼 타입 | 장고의 클래스 변수 | 장고의 필드 클래스 |
| :------------ | :-------- | :----------------- | :----------------- |
| id            | integer   | (id)               | (PK는 장고에서 자동 생성해줌) |
| question_text | varchar(200) | question_text   | models.CharField(max_legnth=200) |
| pub_date      | datetime  | pub_date           | models.DateTimeField('date published') |

* Choice 테이블 컬럼과 클래스 변수 간 매핑

| 테이블 컬럼명 | 컬럼 타입 | 장고의 클래스 변수 | 장고의 필드 클래스 |
| :------------ | :-------- | :----------------- | :----------------- |
| id            | integer   | (id)               | (PK는 장고에서 자동 생성해줌) |
| choice_text   | varchar(200) | choice_text    | models.CharField(max_legnth=200) |
| votes         | integer   | votes              | models.IntegerField(default=0) |
| question_id   | integer   | question           | models. ForeignKey(Question) |

## 2. Admin 사이트에 테이블 반영
이제 models.py 에서 정의한 테이블도 등록해보자

In [ ]:
# admin.py
from django.contrib import admin
from polls.models import Question, Choice

admin.site.register(Question)
admin.stie.register(Choice)

여기까지 하고 manage.py를 실행시키면 된다.

## 3. DB 변경사항 반영
앞 서 DB에 변경사항이 생기면 이를 실제로 반영해줘야 한다고 언급했다. 다음 명령으로 반영하자

```
>python manage.py makemigrations
>python manage.py migrate
```
migrations는 테이블의 변경사항을 알려주는 정보이다. 물리적으로는 애플리케이션 디렉토리별로 존재하며 이 파일들을 이용해 migrate 명령으로 DB에 테이블을 만든다.

마지막으로 admin사이트에 접속하면 변경사항이 모두 적용되었음을 확인할 수 있다.

# 애플리케이션 개발하기 - View 및 Template 코딩

polls 애플리케이션은 3개의 페이지가 필요하다. 각 페이지를 보여주기 위한 뷰와 템플릿을 코딩하자.\
이를 위해서는 요청부터 응답까지의 처리 흐름에 대한 로직이 설계되어야한다. 

## 1. URLconf 코딩

서로 관련있는 URL과 뷰 및 템플릿을 매핑하여 설계한다.\
중요한 점은 URL과 뷰는 1:1 관계로(N:1도 가능) 매핑된다. 이러한 매핑을 URLconf라고하며 urls.py에 작성한다.


| URL 패턴      | 뷰 이름   | 뷰가 처리하는 내용 | 
| :------------ | :-------- | :----------------- |
| /polls        | index()   | index.html         |
| /polls/5/     | detail()  | detail.html        | 
| /polls/vote/  | results() | results.html       | 
| /polls/results/  | results() | results.html       | 
| /admin/       | (장고 기능)   | Admin 사이트           |

<center>※ 5는 질문번호의 예시</center>
 
총 5개의 사이트가 필요하다. 그 내용을 기계적으로 urls.py 파일에 코딩하자.


In [ ]:
# urls.py
from django.contrib import admin
from django.urls import path
from polls import views

urlpatterns = [
    path('admin/', admin.site.urls), # 이미 지정되어있다.
    path('polls/', views.index, name='index'),
    path('polls/<int:question_id', views.detail, name='detail'),
    path('polls/<int:question_id>/results/', views.results, name='results'),
    path('polls/<int:question_id>/vote/', views.vote, name='vote'),
]

URL 패턴 매칭은 위에서 아래에서 진행하므로 정의하는 순서에 유의해야한다.\
path 함수는 장고에서 제공하는 함수로, route와 view 2개의 필수인자와 kwargs,name 선택인자를 받는다.
* route : URL 패턴을 표현하는 문자열
* view : URL 스트링이 매칭되면 호출되는 뷰 함수
* kwargs : URL에서 추출된 항목 외에 추가적 인자를 뷰 함수에 전달
* name : 각 URL별로 이름을 붙여준다. 

URL이 매칭되는 과정을 보자\
만약 요청 URL이 /admin/이라면 아래 라인이 매칭된다. 
```python
path('admin/', admin.site.urls)
```
polls/3 이라면 아래와 매칭된다. 
```python
path('polls/<int:question_id', views.detail, name='detail'),```

urls.py 파일은 여러개에 나눠 작성할 수 있다. 즉 URLconf모듈을 계층적으로 구성하는 것인데, 이는 변경도 쉽고 확장도 용이해서 권장되는 방법이다. 


## 2. 뷰 함수 index() 및 템플릿 작성

index.html을 구현하기 위해 뷰 함수와 템플릿 파일을 코딩하자.\
간단히 설명하자면 3개의 질문을 표시하고 선택하는 질문에 따라서 페이지를 이동한다.

먼저 polls 디렉토리에 index.html 파일을 생성한다. 

* 모든 html파일은 templates 밑에 있어야한다!

```html
# index.html
# 템플릿 언어가 사용되었다.

# 이 latest_quesiton_list 객체는 index() 뷰 함수에서 넘겨주는 파라미터이다.
{% if latest_question_list %}
    # 객체를 순회하면서 question_text를 순서 없느 리스트로 화면에 보여준다.(ul, li 태그의 역할)
    # 각 텍스트에 URL 링크를 연결한다.(a href의 역할)
    <ul>
        {% for question in latest_question_list %}
            <li><a href="/polls/{ { question.id } }/">{ { question.qeustion_list } } </a></li>
        {% endfor %}
    </ul>
{% else}
    # 객체 내용이 없다면 아래 문장을 출력한다.
    <p>No polls are available.</p>
{% endif %}
```

중요한 점은 index.html을 코딩하면서 필요한 변수를 찾아야 한다. 변수는 뷰 함수에서 context 변수로 정의해서 템플릿으로 넘겨줘야하기 때문이다.\
예제에서는 질문인 question_text를 화면에 보여줘야 하고, URL링크를 연결하기 위한 question.id가 필요하다.\
이 정보들이 들어이는 Question 객체를 뷰 함수로부터 넘겨받으면 된다. 이를 views.py에 작성하자

In [ ]:
# view.py

# 장고의 단축함수(개발 시 자주 사용되는 기능들의 모음)
from django.shortcuts import render
from polls.models import Question

# request 인자는 필수이다.
def index(request):
    # Question테이블 객체에서 pub_date 컬럼의 역순으로 정렬하여 5개의 최근 객체를 가져온다.
    latest_question_list = Question.objects.all().order_by('-pub_date')[:5]
    # 딕셔너리 타입으로 변수명과 해당 객체를 매핑하여 context변수로 만들고 render에 보낸다.
    context = {'latest_question_list': latest_question_list}
    # render함수는 html에 context변수를 적용하여 최종 html 텍스트를 만드록 이를 담아서 HtttpResponse 객체를 반환한다.
    return render(request, 'polls/index.html', context)


## 3. 뷰 함수 detail()

이제 3개의 질문 중 하나를 선택했을 때, 질문에 대한 답변 항목을 보여주고 투표하도록 화면을 만드는 detail.html을 코딩한다.


```html
# deatail.html
# 질문 제목
<h1>{{ question.question_text }}</h1>

# 에러 시 에러를 출력. 
{% if error_message %}<p><strong>{{ error_message }}</strong></p>{% endif %}

# 폼에 입력된 데이터는 POST 방식으로 보낸다. 
<form action="{% url 'polls:vote' question_id %}" method="post">
    # 폼을 처리하는 경우 보안 측면에서 CSRF(Cross Site Request Forgery) 공격을 주의해야한다. 
    {% csrf_token %}
    # Question 객체의 choice_set 속성 모든 항목
    {% for choice in question.choice_set.all %}
        # 라디오 버튼을 선택하면 POST데이터가 'choice'='3'(choice.id) 형태로 구성되도록 정의
        <input type="radio" name="choice" id="choice{{ forloop.counter }}" value="{{ choice.id }}"/>
        # forloop.counter는 반복횟수를 담고있는 템플릿 변수이다. 
        <label for="choice{{ forloop.counter }}">{{ choice.choice_text }}</label><br />
    {% endfor %}
    # Vote를 클릭하면 폼 데이터가 polls:vote URL로 전송된다.
    <input type="submit" value="Vote" />
</form>
```

Question과 Choice 테이블의 관계는 1:N 관계이며 외래키로 연결되어 있다. 1:N 관계에서는 한 테이블에 연결된 N 테이블의 항목등이라는 의미로 set 속성을 제공한다.\
question.choice_set.all()은 Question 테이블의 question 레코드에 연결된 Choice 테이블 레코드 모두를 뜻한다.\
이제 뷰 함수를 코딩하자

In [ ]:
# views.py
# 이전 코드는 그대로 유지하면서 다음 함수를 추가한다.

# get_object_or_404의 인자는 모델 클래스와 검색 조건이며 조건에 맞지 않으면 404를 발생시킨다
from django.shortcuts import get_object_or_404, render

# question_id 인자가 추가로 생겼다. URL 패턴에서 정규표현식으로 추출한 question_id 파라미터가 인자로 넘어온 것이다.
def detail(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    return render(request, 'polls.detail.html', {'question':question})

## 4. vote() 및 리다이렉션 

vote 뷰 함수의 호출과 연계된 URL은 detail.html 템플릿에서 받는다. 즉, detail 템플릿에 있는 폼을 제출하면 기능에 의해\
/polls/5/vote/와 같은 URL이 POST로 넘어온다. 방금 작성한 detail 템플릿을 보면 확인 할 수 있다.
```html
<form action="{% url 'polls:vote' question_id %}" method="post">
```
또한 urls.py를 보면
```python
path('polls/<int:question_id>/vote/', views.vote, name='vote')
```
위 두 라인에 의해 사용자가 Vote버튼을 누르면 vote() 뷰 함수가 호출된다. 



In [ ]:
# views.py
# 원래 코드에 추가 한다.

# 이번 뷰에서는 리다이렉트 기능이 필요하다
from django.http import HttpResponseRedirect
# url 처리를 위한 함수
from django.urls import reverse
s
def vote(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    try:
        # Choice 테이블을 검색하며 조건은 기본키 = 'choice'를 키로 검색한다.
        selected_choice = question.choice_set.get(pk=request.POST['choice'])
    except (KeyError, Choice.DoesNotExist):
        return render(request, 'polls/detail.html', {
            'question': question,
            'error_message': "선택이 완료되지 않았습니다",
        })
    else:
        selected_choice.votes += 1
        selected_choice.save()
        return HttpResponseRedirect(reverse('polls:results', args=(question.id,)))

* reverse 함수 : URLconf는 일반적으로 URL 스트링과 뷰를 매핑한 각 라인을 URL 패턴이라 하고 이름을 하나씩 부여한다. 그 반대방향인 reverse로 URL 패턴명으로부터 URL 스트링을 구할 수도 있다. 


## 5. results() 및 템플릿

results와 연계된 URL은 votes의 리다이렉트 결과로 받는다. 

In [ ]:
# views.py

def results(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    return render(request, 'polls/results.html', {'question':question})

results 템플릿의 목표는 투표 결과로, 각 질문마다 투표 카운트를 보여주는 화면을 만드는 것이다.


```html
# results.html
<h1>{{ question.quesiton_text }}</h1>

<ul>
    {% for choice in question.choice_set.all %}
        # vote{{ choice.votes|pluralize }}는 votes값에 따라 복수 접미사를(s)를 붙여주는 것이다.
        <li>{{ choice.choice_text }} - {{ choice.votes }} vote{{ choice.votes|pluralize }}</li>
    {% endfor %}
</ul>

<a href="{% url 'polls:detail' question.id %}">다시 투표하기</a>
```

흥미로운 점 하나는 뷰 함수와 템플릿 태그 양쪽에서 모두 URL스트링을 추출할 수 있다는 것이다.\
뷰 함수에서는 reverse()를 사용하며 템플릿에서는 {% url %} 태그를 사용한다.

## 6. 작업 확인하기

지금까지 설계 로직에 따라 필요한 뷰와 템플릿 코딩을 마쳤다.\
이제 runserver해서 http://127.0.0.1:8000/polls/ 에 접속하자.\

만약 No polls are available 한 줄이 출력된다면 정상이다.\
현재는 DB에 데이터가 들어있지 않아서 질문이 없는 빈 페이지로 나타난다.\
데이터 입력은 Admin사이트를 사용하면 편하다.

Admin사이트에서 Questions 테이블 행의 Add 버튼을 클릭하자.\
그리고 Add question 페이지에서 질문을 하나 추가한다.
예제와 같이 내 입맛에 따라 질문 3가지를 만들었다.

| 질문      | Date   | Time  | 
| :------------ | :-------- | :----------------- |
| 부먹 vs 찍먹     | Today   | Now       |
| 라면 끓일 때 스프먼저 vs 면 먼저 | 2018-04-04  | Midnight        | 
| 펩시 vs 코카콜라  | 2020.02.29 | Noo  | 

그 다음으로 Choice에서 선택사항들을 추가한다.

| 질문      | Choice Text   | Votes  | 
| :------------ | :-------- | :----------------- |
| 부먹 vs 찍먹     | 부먹   | 0       |
| 부먹 vs 찍먹     | 찍먹   | 0       |
| 라면 끓일 때 스프먼저 vs 면 먼저 | 스프  | 0        | 
| 라면 끓일 때 스프먼저 vs 면 먼저 | 면  | 0        | 
| 펩시 vs 코카콜라  | 펩시 | 0  | 
| 펩시 vs 코카콜라  | 코카콜라 | 0  | 

이제 다시 http://127.0.0.1:8000/polls/ 로 접속하면 우리가 기대하는 페이지를 볼 수 있다.